<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    


### <font color='navy'> Simulación de procesos financieros. 

**Nombres:**  
    
    Estudiante 1. Alexia Marisol Díaz Verduzco     
    Estudiante 2. Marco Antonio Ochoa Cardenas
    
    
**Fecha:** 27 de noviembre de 2021.

**Expedientes** :  
    
    Estudiante 1. if720500  
    Estudiante 2. if720072 
    
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/AlexiaDiaz/Tarea10.AMDiaz_MOchoa.git

# Tarea 10: Clase 24. Valuación de opciones con barrera


>Realizar la valuación de las opciones barrera para las acciones de Microsoft, siguiendo los siguientes lineamientos:
1. Realizar la valuación para el tipo de opción barrera 'CALL'- down-and-out, usando la suposición de rendimientos normales y usando el método del histograma. Para ambas valuaciones, comparar los resultados de la evaluación obtenida con los resultados obtenidos por la valuación Europea. Como se realizó en clase.
2. Repetir el inciso anterior pero en esta ocasión para el caso de 'PUT'- Up-and-out.

**Nota:** Analice los datos descargados y según el método ya sea `down-and-out` o `Up-and-out` defina un strike price usando la volatility smile vista en la clase de opciones europeas y basado en el spot price y el strike defina una barrera adecuada.

Se habilitará un enlace en canvas para subir un cuaderno de python, con la solución.

**Criterio de revisión.**

> Se te calificará de acuerdo a los resultados finales que reportes, basados en tu análisis.

## Estudiante 1

Código de solución estudiante 1

## Estudiante 2

Código de solución estudiante 2